# Multi-LLM ワークフローの基本

３つのシンプルなマルチLLMワークフローを体験する
これらはコストとレイテンシーをトレードオフにして、タスクのパフォーマンスを向上させる可能性があります

1. プロンプトチェーン (Prompt-Chaining) タスクを一連のサブタスクに分解。前の結果に基づいて次のステップを作成する
2. 並列化 (Parallelization) 独立したサブタスクを複数のLLMに分散し、同時処理を行う。
3. ルーティング(Routing) 入力の特性に基づいて、特化されたLLMパスを動的に選択する。
